In [1]:
import torch

from src.model import get_model
from src.dataset import FrameDataset, split_dataset, get_pandas_dataset
from src.trainer import Trainer
from src.results import prepare_result
from src.transforms import get_transform

In [2]:
baseTrainPath = 'data/train_images/train_images/'
train_dataset_path = 'data/usdc_train.json'

baseValPath = 'data/test_images/test_images/'
val_dataset_path = 'data/test_file_names.json'

In [3]:
normalization = 'local_contrast_normalization'
apply_transforms = False
batch_size = 32

train_pandas_dataset, classes = get_pandas_dataset(train_dataset_path)

frame_dataset = FrameDataset(train_pandas_dataset, baseTrainPath, 
                             transforms=get_transform() if apply_transforms else None,
                             normalization=normalization)
train_dataset, test_dataset = split_dataset(frame_dataset)

pin_memory = torch.cuda.is_available()

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

val_pandas_dataset, _ = get_pandas_dataset(val_dataset_path, val=True)
val_frame_dataset = FrameDataset(val_pandas_dataset, baseValPath,
                                 transforms=get_transform(val=True) if apply_transforms else None,
                                 val=True,
                                 normalization=normalization)
val_index_map = list(val_pandas_dataset['id'])

val_dataloader = torch.utils.data.DataLoader(
    val_frame_dataset,
    batch_size=batch_size,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
autocast = torch.cuda.is_available()

model = get_model(len(classes)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=5, gamma=0.1)
num_epochs = 5

trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    val_dataloader=val_dataloader,
    optimizer=optimizer,
    scheduler=lr_scheduler,
    num_epochs=num_epochs,
    device=device,
    autocast=autocast
)

trainer.train()

result = trainer.eval()
prepare_result(result, 'result.csv', val_index_map)

Test: 100%|██████████| 67/67 [00:29<00:00,  2.30it/s]


Epoch 0: train_loss  0.63, mAP  0.04


Test: 100%|██████████| 67/67 [00:29<00:00,  2.29it/s]


Epoch 1: train_loss  0.70, mAP  0.04


Test: 100%|██████████| 67/67 [00:28<00:00,  2.32it/s]


...

Epoch 48: train_loss  0.77, mAP  0.07


Test: 100%|██████████| 67/67 [00:30<00:00,  2.21it/s]


Epoch 49: train_loss  0.77, mAP  0.07


Validation:   0%|          | 0/188 [00:00<?, ?it/s]


TypeError: Expected input images to be of floating type (in range [0, 1]), but found type torch.uint8 instead